## Visualização do Dataset KITTI

In [ ]:
# Import libraries
import os
import sys
import json
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
%matplotlib inline
cv2.cuda.getCudaEnabledDeviceCount()

In [ ]:
# Load config.json file
config = json.load(open('./config.json'))
KITTI_path = config["KITTI"]["path"]
sequences_path  = KITTI_path + config["KITTI"]["sequences path"]
poses_path      = KITTI_path + config["KITTI"]["poses_path"]
num_sequences   = config["KITTI"]["num_sequences"]
num_poses       = config["KITTI"]["num_poses"]

print("KITTI path: ", KITTI_path)
print("Sequences path: ", sequences_path)
print("Poses path: ", poses_path)
print("Number of sequences: ", num_sequences)
print("Number of poses: ", num_poses)

In [ ]:
# Load poses
poses = [pd.read_csv('{}/{:02}.txt'.format(poses_path, i), delimiter=' ', header=None) for i in range(num_poses)]
poses[0].head()

In [ ]:
# We load the ground truth poses into a list of numpy arrays
# Then, we do the dot product between the poses and (0, 0, 0, 1) to get the 
# postiion of the camera in relation to it's first position (0, 0, 0)

gt = []
for i in range(len(poses)):
    gt.append(np.zeros((len(poses[i]), 3, 4)))
    for j in range(len(poses[i])):
        gt[i][j] = np.array(poses[i].iloc[j]).reshape((3, 4))

gtv = []
for i in range(len(poses)):
    gtv.append(np.zeros((len(poses[i]), 3)))
    for j in range(len(poses[i])):
        gtv[i][j] = gt[i][j].dot(np.array([0,0,0,1]))

print(gt[0][1].dot(np.array([0,0,0,1])))
print(gt[0][1])

In [ ]:
# Trajectory demonstration

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(gtv[0][:,0], gtv[0][:,1], gtv[0][:,2], label='Ground Truth')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(elev = -40, azim = 270)

### Demonstração da relação entre coordenadas em pixel $u, v$ e coordenadas em metros $x, y, z$.
- $\lambda = $ escala de profundidade.
- $K = $ matriz de calibração intrínseca.
- $R = $ matriz de rotação.
- $t = $ vetor de translação.

![Projection Matrix](../resources/intrinsic_extrinsic.png)

In [ ]:
cb = pd.read_csv(sequences_path + '/00/calib.txt', delimiter=' ', header=None, index_col=0)
cb.head()

In [ ]:
ti = pd.read_csv(sequences_path + '/00/times.txt', delimiter=' ', header=None, index_col=0)
ti.head()

In [ ]:
# Read calibration and times files for each sequence
camera_calib = []
time_stamps  = []
for i in range(num_sequences):
    cb = pd.read_csv(sequences_path + '/{:02}/calib.txt'.format(i), delimiter=' ', header=None, index_col=0).loc['P0:']
    ti = pd.read_csv(sequences_path + '/{:02}/times.txt'.format(i), delimiter=' ', header=None)
    camera_calib.append(np.array(cb).reshape((3, 4))[0:3, 0:3])
    time_stamps.append(np.array(ti))
camera_calib = np.array(camera_calib)

In [ ]:
import main
seq_name = '00'
seq_idx  = int(seq_name)
preffered_direction = 2
main.run_KITTI_visual_odometry(sequences_path, seq_name, seq_idx, gt[seq_idx], camera_calib[seq_idx], time_stamps[seq_idx], preffered_direction)